In [48]:
# Cell 1 – Import libraries and connect to my car sales database

import sqlite3
import random
import datetime
from faker import Faker

db_path = r"C:\Yarra\Cars_database.db"

# open connection
connection = sqlite3.connect(db_path)
cursor = connection.cursor()

# make sure foreign keys work
cursor.execute("PRAGMA foreign_keys = ON;")

print("Connected to:", db_path)


Connected to: C:\Yarra\Cars_database.db


In [49]:
# Cell 2 – Drop old car tables if they exist (fresh start)

tables_to_drop = [
    "test_drives",
    "payments",
    "sales_orders",
    "cars_inventory",
    "customers",
    "dealerships"
]

for tbl in tables_to_drop:
    cursor.execute(f"DROP TABLE IF EXISTS {tbl};")
    print("Dropped:", tbl)

connection.commit()
print("\nOld tables removed.\n")


Dropped: test_drives
Dropped: payments
Dropped: sales_orders
Dropped: cars_inventory
Dropped: customers
Dropped: dealerships

Old tables removed.



In [50]:
# Cell 3 – Create final car sales schema

schema_sql = """
CREATE TABLE dealerships (
    dealer_id        INTEGER PRIMARY KEY,
    dealer_name      TEXT NOT NULL,
    city             TEXT NOT NULL,
    country          TEXT NOT NULL,
    phone_number     TEXT,
    is_active        INTEGER NOT NULL CHECK (is_active IN (0,1))
);

CREATE TABLE customers (
    customer_id          INTEGER PRIMARY KEY,
    first_name           TEXT NOT NULL,
    last_name            TEXT NOT NULL,
    email                TEXT NOT NULL UNIQUE,
    phone_number         TEXT,
    city                 TEXT NOT NULL,
    registration_date    TEXT NOT NULL,   -- interval (date)
    credit_band          TEXT NOT NULL    -- ordinal band
        CHECK (credit_band IN ('Poor','Fair','Good','Very Good','Excellent'))
);

CREATE TABLE cars_inventory (
    car_id             INTEGER PRIMARY KEY,
    dealer_id          INTEGER NOT NULL,
    vin                TEXT NOT NULL UNIQUE,
    make               TEXT NOT NULL,      -- nominal
    model              TEXT NOT NULL,
    year_of_make       INTEGER NOT NULL CHECK (year_of_make BETWEEN 1990 AND 2025),
    fuel_type          TEXT NOT NULL CHECK (fuel_type IN ('Petrol','Diesel','Hybrid','Electric')),
    transmission       TEXT NOT NULL CHECK (transmission IN ('Manual','Automatic')),
    body_type          TEXT NOT NULL,
    colour             TEXT NOT NULL,
    mileage_km         REAL NOT NULL CHECK (mileage_km >= 0),   -- ratio
    engine_size_litre  REAL CHECK (engine_size_litre > 0),      -- ratio
    list_price         REAL NOT NULL CHECK (list_price > 0),    -- ratio
    is_new             INTEGER NOT NULL CHECK (is_new IN (0,1)),
    is_available       INTEGER NOT NULL CHECK (is_available IN (0,1)),
    FOREIGN KEY (dealer_id) REFERENCES dealerships(dealer_id)
);

CREATE TABLE sales_orders (
    sale_id          INTEGER PRIMARY KEY,
    car_id           INTEGER NOT NULL,
    customer_id      INTEGER NOT NULL,
    dealer_id        INTEGER NOT NULL,
    sale_date        TEXT NOT NULL,      -- interval
    sale_price       REAL NOT NULL CHECK (sale_price > 0),
    discount_amount  REAL NOT NULL DEFAULT 0 CHECK (discount_amount >= 0),
    payment_method   TEXT NOT NULL CHECK (payment_method IN ('Cash','Card','Finance','Bank Transfer')),
    warranty_years   INTEGER NOT NULL CHECK (warranty_years BETWEEN 0 AND 7),
    satisfaction     INTEGER CHECK (satisfaction BETWEEN 1 AND 5),  -- ordinal
    FOREIGN KEY (car_id)      REFERENCES cars_inventory(car_id),
    FOREIGN KEY (customer_id) REFERENCES customers(customer_id),
    FOREIGN KEY (dealer_id)   REFERENCES dealerships(dealer_id),
    CONSTRAINT unique_car_sale UNIQUE (car_id)
);

-- composite primary key for assignment
CREATE TABLE payments (
    sale_id          INTEGER NOT NULL,
    instalment_no    INTEGER NOT NULL,
    payment_date     TEXT NOT NULL,
    amount_paid      REAL NOT NULL CHECK (amount_paid > 0),
    payment_channel  TEXT NOT NULL CHECK (payment_channel IN ('Cash','Card','Online','Cheque')),
    PRIMARY KEY (sale_id, instalment_no),
    FOREIGN KEY (sale_id) REFERENCES sales_orders(sale_id)
);

CREATE TABLE test_drives (
    test_drive_id    INTEGER PRIMARY KEY,
    car_id           INTEGER NOT NULL,
    customer_id      INTEGER NOT NULL,
    dealer_id        INTEGER NOT NULL,
    drive_date       TEXT NOT NULL,
    feedback_score   INTEGER CHECK (feedback_score BETWEEN 1 AND 5), -- ordinal
    went_to_purchase INTEGER NOT NULL CHECK (went_to_purchase IN (0,1)),
    FOREIGN KEY (car_id)      REFERENCES cars_inventory(car_id),
    FOREIGN KEY (customer_id) REFERENCES customers(customer_id),
    FOREIGN KEY (dealer_id)   REFERENCES dealerships(dealer_id)
);
"""

cursor.executescript(schema_sql)
connection.commit()

print("New schema created.\n")

cursor.execute("SELECT name FROM sqlite_master WHERE type='table' AND name NOT LIKE 'sqlite_%';")
print("Tables in this DB:")
for row in cursor.fetchall():
    print(" -", row[0])


New schema created.

Tables in this DB:
 - dealerships
 - customers
 - cars_inventory
 - sales_orders
 - payments
 - test_drives


In [51]:
# Cell 4 – Setup Faker, lists, and helper functions

fake = Faker("en_GB")  # UK style data

credit_bands = ["Poor", "Fair", "Good", "Very Good", "Excellent"]

makes_models = {
    "Toyota":   ["Corolla", "Yaris", "RAV4", "Camry"],
    "BMW":      ["3 Series", "5 Series", "X1", "X3"],
    "Audi":     ["A3", "A4", "Q3", "Q5"],
    "Ford":     ["Fiesta", "Focus", "Kuga", "Puma"],
    "Tesla":    ["Model 3", "Model Y"],
    "Mercedes": ["A-Class", "C-Class", "GLA", "GLC"]
}

body_types = ["Hatchback", "Sedan", "SUV", "Coupe", "Estate"]
fuel_types = ["Petrol", "Diesel", "Hybrid", "Electric"]
transmissions = ["Manual", "Automatic"]
payment_methods = ["Cash", "Card", "Finance", "Bank Transfer"]
payment_channels = ["Cash", "Card", "Online", "Cheque"]

def random_reg_date():
    """Random registration date in last 6 years."""
    return fake.date_between(start_date="-6y", end_date="today").isoformat()

def random_sale_date():
    """Random sale date in last 5 years."""
    return fake.date_between(start_date="-5y", end_date="today").isoformat()

def random_mileage(year):
    """Mileage increases for older cars."""
    if year >= 2022:
        return round(random.uniform(0, 40000), 1)
    return round(random.uniform(40000, 200000), 1)

print("Faker and helpers ready.")


Faker and helpers ready.


In [52]:
# Cell 5 – Insert fake dealerships

dealership_rows = []
num_dealers = 15

for dealer_id in range(1, num_dealers + 1):
    dealer_name = fake.company()
    city = fake.city()
    country = "UK"
    phone = fake.phone_number()
    is_active = random.choice([0, 1])

    dealership_rows.append(
        (dealer_id, dealer_name, city, country, phone, is_active)
    )

cursor.executemany("""
    INSERT INTO dealerships (
        dealer_id, dealer_name, city, country, phone_number, is_active
    ) VALUES (?, ?, ?, ?, ?, ?);
""", dealership_rows)

connection.commit()
print("Dealerships inserted:", len(dealership_rows))


Dealerships inserted: 15


In [53]:
# Cell 6 – Insert fake customers (1500+ rows)

customer_rows = []
num_customers = 1500

for customer_id in range(1, num_customers + 1):
    first_name = fake.first_name()
    last_name = fake.last_name()
    email = f"{first_name.lower()}.{last_name.lower()}{customer_id}@example.com"
    phone = fake.phone_number()
    city = fake.city()
    reg_date = random_reg_date()
    band = random.choice(credit_bands)

    customer_rows.append(
        (customer_id, first_name, last_name, email, phone, city, reg_date, band)
    )

cursor.executemany("""
    INSERT INTO customers (
        customer_id, first_name, last_name, email,
        phone_number, city, registration_date, credit_band
    ) VALUES (?, ?, ?, ?, ?, ?, ?, ?);
""", customer_rows)

connection.commit()
print("Customers inserted:", len(customer_rows))


Customers inserted: 1500


In [54]:
# Cell 7 – Insert fake cars into inventory

cursor.execute("SELECT dealer_id FROM dealerships;")
dealer_ids = [row[0] for row in cursor.fetchall()]

car_rows = []
num_cars = 600
car_id = 1
used_vins = set()

for _ in range(num_cars):
    dealer_id = random.choice(dealer_ids)
    make = random.choice(list(makes_models.keys()))
    model = random.choice(makes_models[make])

    year = random.randint(2005, 2025)
    fuel = random.choice(fuel_types)
    gearbox = random.choice(transmissions)
    body = random.choice(body_types)
    colour = fake.color_name()

    mileage = random_mileage(year)
    is_new = 0 if mileage > 500 else 1

    engine_size = round(random.uniform(1.0, 4.0), 1)
    list_price = float(random.randint(8000, 80000))

    vin = fake.bothify("??#####??###").upper()
    while vin in used_vins:
        vin = fake.bothify("??#####??###").upper()
    used_vins.add(vin)

    available = 1  # will turn 0 for sold cars

    car_rows.append((
        car_id, dealer_id, vin, make, model, year,
        fuel, gearbox, body, colour,
        mileage, engine_size, list_price,
        is_new, available
    ))
    car_id += 1

cursor.executemany("""
    INSERT INTO cars_inventory (
        car_id, dealer_id, vin, make, model, year_of_make,
        fuel_type, transmission, body_type, colour,
        mileage_km, engine_size_litre, list_price,
        is_new, is_available
    ) VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?);
""", car_rows)

connection.commit()
print("Cars inserted:", len(car_rows))


Cars inserted: 600


In [55]:
# Cell 8 – Insert sales orders (around 70% cars sold)

cursor.execute("SELECT car_id, dealer_id, list_price FROM cars_inventory;")
all_cars = cursor.fetchall()
random.shuffle(all_cars)

sold_limit = int(len(all_cars) * 0.7)
sold_cars = all_cars[:sold_limit]

cursor.execute("SELECT customer_id FROM customers;")
customer_ids = [row[0] for row in cursor.fetchall()]

sales_rows = []
sale_id = 1

for car_id_val, dealer_id_val, list_price in sold_cars:
    customer_id = random.choice(customer_ids)
    sale_date = random_sale_date()

    max_disc = list_price * 0.20
    discount = round(random.uniform(0, max_disc), 2)
    sale_price = round(list_price - discount, 2)

    method = random.choice(payment_methods)
    warranty = random.randint(0, 5)
    satisfaction = random.randint(1, 5)

    sales_rows.append((
        sale_id, car_id_val, customer_id, dealer_id_val,
        sale_date, sale_price, discount,
        method, warranty, satisfaction
    ))
    sale_id += 1

cursor.executemany("""
    INSERT INTO sales_orders (
        sale_id, car_id, customer_id, dealer_id,
        sale_date, sale_price, discount_amount,
        payment_method, warranty_years, satisfaction
    ) VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?);
""", sales_rows)

# update inventory availability
sold_ids_only = [row[0] for row in sold_cars]
cursor.executemany("""
    UPDATE cars_inventory
    SET is_available = 0
    WHERE car_id = ?;
""", [(cid,) for cid in sold_ids_only])

connection.commit()
print("Sales inserted:", len(sales_rows))
print("Sold cars marked as unavailable.")


Sales inserted: 420
Sold cars marked as unavailable.


In [56]:
# Cell 9 – Insert payments with 1–4 instalments per sale

cursor.execute("SELECT sale_id, sale_date, sale_price FROM sales_orders;")
sale_info = cursor.fetchall()

payment_rows = []

for sale_id_val, sale_date_str, sale_price in sale_info:
    instalments = random.randint(1, 4)
    remaining = sale_price
    base_date = datetime.date.fromisoformat(sale_date_str)

    for inst_no in range(1, instalments + 1):
        if inst_no == instalments:
            amount = round(remaining, 2)
        else:
            avg = sale_price / instalments
            amount = round(avg + random.uniform(-100, 100), 2)
            remaining -= amount

        if amount <= 0:
            # if something weird, force positive value
            amount = round(abs(amount) + 50, 2)

        pay_date = (base_date + datetime.timedelta(days=random.randint(1, 90))).isoformat()
        channel = random.choice(payment_channels)

        payment_rows.append((
            sale_id_val, inst_no, pay_date, amount, channel
        ))

cursor.executemany("""
    INSERT INTO payments (
        sale_id, instalment_no, payment_date,
        amount_paid, payment_channel
    ) VALUES (?, ?, ?, ?, ?);
""", payment_rows)

connection.commit()
print("Payments inserted:", len(payment_rows))


Payments inserted: 1047


In [57]:
# Cell 10 – Insert test drive history

cursor.execute("SELECT car_id, dealer_id FROM cars_inventory;")
cars_for_tests = cursor.fetchall()

cursor.execute("SELECT customer_id FROM customers;")
customer_ids = [row[0] for row in cursor.fetchall()]

test_drive_rows = []
test_drive_id = 1

for car_id_val, dealer_id_val in cars_for_tests:
    drive_times = random.randint(0, 5)
    for _ in range(drive_times):
        cust_id = random.choice(customer_ids)
        drive_date = fake.date_between(start_date="-6y", end_date="today").isoformat()
        feedback = random.randint(1, 5)
        went_purchase = random.choice([0, 0, 0, 1])

        test_drive_rows.append((
            test_drive_id, car_id_val, cust_id, dealer_id_val,
            drive_date, feedback, went_purchase
        ))
        test_drive_id += 1

cursor.executemany("""
    INSERT INTO test_drives (
        test_drive_id, car_id, customer_id, dealer_id,
        drive_date, feedback_score, went_to_purchase
    ) VALUES (?, ?, ?, ?, ?, ?, ?);
""", test_drive_rows)

connection.commit()
print("Test drives inserted:", len(test_drive_rows))


Test drives inserted: 1409


In [58]:
# Cell 11 – Show final row counts and close DB

tables = [
    "dealerships",
    "customers",
    "cars_inventory",
    "sales_orders",
    "payments",
    "test_drives"
]

print("Final row counts in car sales DB:")
for name in tables:
    cursor.execute(f"SELECT COUNT(*) FROM {name};")
    count = cursor.fetchone()[0]
    print(f"{name:15} -> {count}")

connection.close()
print("\nDatabase connection closed.")


Final row counts in car sales DB:
dealerships     -> 15
customers       -> 1500
cars_inventory  -> 600
sales_orders    -> 420
payments        -> 1047
test_drives     -> 1409

Database connection closed.
